<hr>

 ### PS3 ###

 
<hr>

In [1]:
import numpy as np
import sympy 
from sympy import *

$$ \begin{equation}


    U_t = \begin{bmatrix} k_{t+1} \\ h_t \end{bmatrix} \quad\quad \quad



    X_t = \begin{bmatrix}  

            X_{1t} \\ X_{2t} \\ X_{3t} 
            
          \end{bmatrix} 

        = \begin{bmatrix} 

             \begin{bmatrix} 
                1 \\ k_{t} 
             \end{bmatrix} 

                \\

            \begin{bmatrix} 
                ln({z_{t}}) \\ \tau_{ct} \\ \tau_{ht} \\ \tau_{dt} \\ \tau_{pt} \\ log(g_t)  
            \end{bmatrix}

                \\
                    
            \begin{bmatrix}
                K_t \\ H_t \\ \kappa_t
            \end{bmatrix} 
                    
           \end{bmatrix}\\

\end{equation} 
$$

In [2]:
'''
This class defines the jacobian and the Hessian for any given differentiable function.

Inputs: f = Function.
        x = np.array of values around which we are going to differentiate.

Outputs: The Jacobian as a column vector.
         The Hessian Matrix

....... Example .......

> def g(x, y, z):
>    return x**2 + y**3 + z**8 + x**2 * z + y** x

> G = Derivatives(g, np.array([1, 1, 1]))
> G.J, G.H

> (array([[4.002     ],
>        [4.003001  ],
>        [9.02805607]]),
> array([[4.00000000e+00, 1.00050033e+00, 2.00100000e+00],
>        [1.00050033e+00, 6.00600000e+00, 8.88121576e-10],
>        [2.00100000e+00, 8.88121576e-10, 5.63369817e+01]]))

'''

class Derivatives:
    
    def __init__(self, f, x):
        self.f = f
        self.x = x
        self.J = self.Compute_derivatives()[0]
        self.H = self.Compute_derivatives()[1]

    def Compute_derivatives(self):
        x_ss = self.x
        dim = int(np.shape(x_ss)[0])
        Δ = np.maximum(np.ones([1, dim]) * 10e-8, 10.e-4 * np.abs(x_ss))
        J = np.zeros([dim,1])
        H_aux = np.zeros([dim,np.shape(x_ss)[0]])
        for i in range(dim):
            e_1 = np.zeros([1,dim])
            e_1[0, i] = 1
            y = x_ss + Δ * e_1
            J[i,0] = (self.f(*y[0,:]) - self.f(*x_ss)) / (Δ @ e_1.T)
            for j in range(dim):
                e_2 = np.zeros([1,dim])
                e_2[0, j] = 1
                z_1 = x_ss + Δ * e_1 + Δ * e_2
                H_aux[i,j] = ((self.f(*z_1[0,:]) - self.f(*y[0,:])) / (Δ @ e_1.T) - J[i,0]) / (Δ @ e_2.T)
        return J, (0.5*H_aux + 0.5*H_aux.T)


In [3]:
β = 0.9
γ_n = 0.1
γ_z= 0.1
θ= 0.7
ψ= 0.3
δ=0.9
ϕ_g=0.01

#Compute the steady state

λ_1 = ((((1+γ_z)/β) - (1-δ)) / (θ) )**( 1 / (1-θ) )
λ_2 = ((1-ϕ_g) * λ_1**(1-θ) + (1 - δ - (1+γ_z) * (1+γ_n)))
th_ss = 0
tc_ss = 0
tp_ss = 0
td_ss = 0 
lnz_ss = 1
w_ss = (1-θ) * (λ_1)**(-θ)
k_ss = w_ss / (w_ss * λ_1 + ψ * λ_2) 
kp_ss = k_ss
h_ss = λ_1 * k_ss
y_ss = k_ss**(θ) * h_ss**(1-θ)
r_ss = θ * (y_ss/k_ss)
K_ss = k_ss
H_ss = h_ss
c_ss = λ_2 * k_ss
g_ss = ϕ_g * y_ss
κ_ss = -g_ss
lng_ss = np.log(g_ss)

In [4]:
def U(kp, h, k, lnz, tc, th, td, tp, K, H, κ):

    r = θ * K**(θ-1) * (np.exp(lnz)*H)**(1-θ)
    w = (1-θ) * K**(θ) * (np.exp(lnz)**(1-θ)) * H**(-θ)
    x = (1+γ_n) * (1+γ_n) * kp - (1-δ) * k 
    c = r * k + w * h + κ - th * w * h - tp * (r * k - δ * k) - td * (r * k - x - tp * (r * k - δ * k)) - x

    return np.log(c) + ψ * np.log(1-h)

In [5]:
e_ss = np.array([kp_ss, h_ss, k_ss, lnz_ss, tc_ss, th_ss, td_ss, tp_ss, K_ss, H_ss, κ_ss])
#U(kp_ss, h_ss, k_ss, lnz_ss, tc_ss, th_ss, td_ss, tp_ss, K_ss, H_ss, κ_ss)
Aux = Derivatives(U, e_ss)


In [6]:
kp, h, k, lnz, tc, th, td, tp, K, H, κ = symbols('kp h k lnz tc th td tp K H κ')
x = [kp, h, k, lnz, tc, th, td, tp, K, H, κ]
x_ss = np.array([kp_ss, h_ss, k_ss, lnz_ss, tc_ss, th_ss, td_ss, tp_ss, K_ss, H_ss, κ_ss])
Taylor = U(*x_ss) + Aux.J.T[0,:] @  (x-x_ss) + 0.5 * ((x-x_ss).T @ Aux.H[:,:] @ (x-x_ss))
coeff = sympy.expand(Taylor).as_coefficients_dict()
for k in coeff.keys():
    coeff[k] = float(coeff[k])  

In [20]:
Q1 = np.array([coeff[1], 0.5 * coeff[1*k], 0.5 * coeff[1*lnz], 0.5 * coeff[1*tc], 0.5 * coeff[1*th],0.5 * coeff[1*td],0.5 * coeff[1*tp], 0, 0.5 * coeff[1*K], 0.5 * coeff[1*H], 0.5*coeff[1*κ]])
Q2 = np.array([coeff[k**2], 0.5 * coeff[k*lnz], 0.5 * coeff[k*tc], 0.5 * coeff[k*th],0.5 * coeff[k*td],0.5 * coeff[k*tp], 0, 0.5 * coeff[k*K], 0.5 * coeff[k*H], 0.5*coeff[k*κ]])
Q3 = np.array([coeff[lnz**2], 0.5 * coeff[lnz*tc], 0.5 * coeff[lnz*th],0.5 * coeff[lnz*td],0.5 * coeff[lnz*tp], 0, 0.5 * coeff[lnz*K], 0.5 * coeff[lnz*H], 0.5*coeff[lnz*κ]])
Q4 = np.array([coeff[tc**2], 0.5 * coeff[tc*th],0.5 * coeff[tc*td],0.5 * coeff[tc*tp], 0, 0.5 * coeff[tc*K], 0.5 * coeff[tc*H], 0.5*coeff[tc*κ]])
Q5 = np.array([coeff[th**2],0.5 * coeff[th*td],0.5 * coeff[th*tp], 0, 0.5 * coeff[th*K], 0.5 * coeff[th*H], 0.5*coeff[th*κ]])
Q6 = np.array([coeff[td**2],0.5 * coeff[td*tp], 0, 0.5 * coeff[td*K], 0.5 * coeff[td*H], 0.5*coeff[td*κ]])
Q7 = np.array([coeff[tp**2], 0, 0.5 * coeff[tp*K], 0.5 * coeff[tp*H], 0.5*coeff[tp*κ]])
Q8 = np.array([0, 0, 0, 0])
Q9 = np.array([coeff[K**2], 0.5 * coeff[K*H], 0.5*coeff[K*κ]])
Q10 = np.array([coeff[H**2], 0.5*coeff[K*κ]])
Q11 = np.array([coeff[κ**2]])

In [41]:
Q = np.zeros((11,11))
giga_Q = [Q1, Q2, Q3, Q4, Q5, Q6, Q7, Q8, Q9, Q10, Q11]
for i in range(11):
    Q[i,i:]= giga_Q[i]
    Q[i:, i ] = giga_Q[i].T

In [38]:
Q = np.zeros((11,11))
Q[0,:] = Q1
Q[:,0] = Q1.T
Q[1,1:]= Q2
Q[1:, 1] = Q2.T
Q

array([[-5.15379800e+00, -1.77575239e+00,  4.13505425e-01,
        -4.77191551e-12, -4.13316562e-01, -9.92084921e-02,
        -2.61786075e-01,  0.00000000e+00,  1.82118019e-03,
         1.13110820e-03,  7.90147742e+00],
       [-1.77575239e+00,  1.44000000e+02,  1.44000000e+02,
         1.44000000e+02,  1.44000000e+02,  1.44000000e+02,
         1.44000000e+02,  1.44000000e+02,  1.44000000e+02,
         1.44000000e+02,  1.44000000e+02],
       [ 4.13505425e-01,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00],
       [-4.77191551e-12,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00],
       [-4.13316562e-01,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
  

In [46]:
np.matrix(Q)
Q.view()

array([[-5.15379800e+00, -1.77575239e+00,  4.13505425e-01,
        -4.77191551e-12, -4.13316562e-01, -9.92084921e-02,
        -2.61786075e-01,  0.00000000e+00,  1.82118019e-03,
         1.13110820e-03,  7.90147742e+00],
       [-1.77575239e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00],
       [ 4.13505425e-01,  0.00000000e+00, -1.01672075e-01,
         0.00000000e+00,  1.01720854e-01, -9.68936044e-02,
        -3.36597420e-02,  0.00000000e+00, -2.22596448e-04,
        -4.61463815e-05, -1.88428978e+00],
       [-4.77191551e-12,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00, -1.86264515e-09],
       [-4.13316562e-01,  0.00000000e+00,  1.01720854e-01,
         0.00000000e+00, -1.77635684e-01, -1.11022302e-01,
  

In [74]:
R = np.array([ [coeff[kp**2], 0.5*coeff[kp*h]] , [0.5*coeff[kp*h], coeff[h**2]] ])

In [65]:
W = 0.5*np.array([  [coeff[kp*1], coeff[h*1]],\
                    [coeff[kp*k], coeff[h*k]],\
                    [coeff[kp*lnz], coeff[h*lnz]],\
                    [coeff[kp*tc], coeff[h*tc]],\
                    [coeff[kp*th], coeff[h*th]],\
                    [coeff[kp*td], coeff[h*td]],\
                    [coeff[kp*tp], coeff[h*tp]],\
                    [0, 0],\
                    [coeff[kp * K], coeff[h * K]],\
                    [coeff[kp * H], coeff[h * H]],\
                    [coeff[kp * κ], coeff[h * κ]]])

W_y = W[0:8,:]
W_z = W[8:,:]
W, W_y, W_z

(array([[-9.56992594e+00,  2.49853804e+00],
        [ 0.00000000e+00,  0.00000000e+00],
        [ 2.28268581e+00, -1.31912323e-01],
        [ 0.00000000e+00,  0.00000000e+00],
        [-2.28376767e+00, -1.51899442e-01],
        [ 2.22738177e+00,  1.58163396e-01],
        [-2.16285341e+00,  2.40205634e-01],
        [ 0.00000000e+00,  0.00000000e+00],
        [ 4.99761930e-03,  1.77559524e+00],
        [ 1.03604203e-03, -3.68164899e-01],
        [ 2.20127243e+01, -2.44473266e+00]]),
 array([[-9.56992594,  2.49853804],
        [ 0.        ,  0.        ],
        [ 2.28268581, -0.13191232],
        [ 0.        ,  0.        ],
        [-2.28376767, -0.15189944],
        [ 2.22738177,  0.1581634 ],
        [-2.16285341,  0.24020563],
        [ 0.        ,  0.        ]]),
 array([[ 4.99761930e-03,  1.77559524e+00],
        [ 1.03604203e-03, -3.68164899e-01],
        [ 2.20127243e+01, -2.44473266e+00]]))

In [54]:
A11 = np.array([[1, 0], [0, 0]])
A12 = np.array([[0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0]])
A12 = np.array([[0, 0, 0], [0, 0, 0]])
A22 = np.zeros((6,6))
ρ, ρ_c, ρ_h, ρ_d, ρ_p, ρ_g = 0.8, 0.8, 0.8, 0.8, 0.8, 0.8
giga_ρ = [ρ, ρ_c, ρ_h, ρ_d, ρ_p, ρ_g]
for i in range(6):
    A22[i,i]=giga_ρ[i]
A23 = np.zeros((6,3))

In [91]:
B = np.zeros((11,2))
B[1,0] = 1
B_y = B[0:8,:]
B, B_y

(array([[0., 0.],
        [1., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.]]),
 array([[0., 0.],
        [1., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.]]))

In [57]:
A_y = np.diag([1, 0, ρ, ρ_c, ρ_h, ρ_d, ρ_p, ρ_g])
A_z = np.zeros((8, 3))

We need to express $\kappa_{t}$ as a function of the variables.

In [67]:
Θ = np.zeros((3, 8))
Θ[0,1]  = 1
Θ[2,0]  = -g_ss * (1 - np.log(g_ss))
Θ[2,-1] = -g_ss

In [71]:
Ψ = np.zeros((3,2))
Ψ[1,1] = 1


array([[0., 0.],
       [0., 1.],
       [0., 0.]])

Normalization

In [124]:
Qtilda = Q - W @ np.linalg.inv(R) @ W.T
Qtilda_y = Qtilda[0:8,0:8]
Qtilda_z = Qtilda[0:8,8:]
Btilda = np.sqrt(β) * B
Atilda_y = np.sqrt(β) * ( A_y - B_y @ np.linalg.inv(R) @ W_y.T) 
Atilda_z = np.sqrt(β) * (A_z - B_y @ np.linalg.inv(R) @ W_z.T)
Btilda_y = np.sqrt(β) * B_y
Θtilda = np.linalg.inv((np.identity(3) + Ψ @ np.linalg.inv(R) @ W_z.T)) @ (Θ - Ψ @ np.linalg.inv(R) @ W_y.T)
Ψtilda = np.linalg.inv((np.identity(3) + Ψ @ np.linalg.inv(R) @ W_z.T)) @ Ψ

In [126]:
Ahat = Atilda_y + Atilda_z @ Θtilda
Qhat = Qtilda_y + Qtilda_z @ Θtilda
Bhat = Btilda_y + Atilda_z @ Ψtilda
Abar = Atilda_y - Btilda_y @  np.linalg.inv(R) @ Ψtilda.T @ Qtilda_z.T

In [192]:
i=0
P0 = np.identity(8)
P1 = np.zeros((8,8))
while i<1000:
    P1 = Qhat + Abar.T @ np.linalg.inv(np.linalg.inv(P0) + Bhat @ np.linalg.inv(R) @ Btilda_y.T) @ Ahat
    if np.max(np.abs(P1-P0))>10e-5:
        P0 = P1.copy()
        i += 1
    else:
        print("Convergence")
        break

Convergence


array([[-2.20499155e+00, -1.04545042e+00, -1.55789793e+00,
        -1.70425554e-11,  1.14306532e+00, -2.89413565e+00,
         2.10967108e+00,  7.09002217e-03],
       [-1.77575239e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00],
       [-1.83892575e+00,  6.99972891e-02,  2.42493746e-01,
         3.27027932e-23, -2.68911646e-01,  2.68661940e-01,
        -5.25677988e-01, -2.51355784e-04],
       [ 1.01693014e-10,  1.47312288e-23,  3.24484487e-22,
         1.07568783e-18, -2.76695240e-22,  6.66104018e-22,
        -2.65557727e-22,  1.12545177e-11],
       [ 3.27906516e+00, -1.50248172e+00, -4.60974589e-01,
        -6.66684518e-23,  3.32270374e-01, -9.96284086e-01,
         1.72470357e-01,  5.39601922e-03],
       [-5.21730963e+00,  1.50250113e+00,  4.60979228e-01,
         9.33314953e-23, -1.00168256e+00,  5.13859160e-01,
        -2.66425087e-01, -2.36063522e-02],
       [ 6.32224033e-01,  1.269063

In [193]:
Hup = np.concatenate((np.linalg.inv(Ahat), np.linalg.inv(Ahat) @ Bhat @ np.linalg.inv(R) @ Btilda_y.T), axis=1)
Hdown = np.concatenate((Qhat @ np.linalg.inv(Ahat), Qhat @ np.linalg.inv(Ahat) @ Bhat @ np.linalg.inv(R) @ Btilda_y.T + Abar), axis=1)
HAM = np.concatenate((Hup, Hdown), axis=0) 

In [194]:
λ, v = np.linalg.eig(HAM)
index = np.where(np.sqrt(λ.real**2 + λ.imag**2)>1, 1, 0)
Λ = np.diag(λ[(index)==True])
V = np.empty([shape(HAM)[0],shape(HAM)[0]])
aux_1 = 0
aux_2 = 0 
for i in range(shape(HAM)[0]):
    if index[i] == 1:
        V[:, aux_1] = v[:, i]
        aux_1 += 1 
    if index[i] == 0:
        V[:, -1-aux_2] = v[:, i]
        aux_2 += 1
V_11 = V[0:8, 0:8]
V_21 = V[8:16, 0:8]
P_Vaughans = V_21 @ np.linalg.inv(V_11)

In [195]:
P_Vaughans- P1

array([[-5.39076143e+00, -6.00288832e-05, -5.07698509e-02,
        -3.96333925e-13,  4.09726845e-02, -8.35280553e-02,
         5.82589260e-02,  4.30737517e-04],
       [ 1.27251811e+00,  2.65061854e-02,  5.04291027e-01,
         3.96130976e-12, -4.06026619e-01,  8.31436539e-01,
        -5.79447234e-01, -4.23102285e-03],
       [ 5.20846384e-01,  3.17831059e-06,  1.79568275e-03,
         1.40189660e-14, -1.44912733e-03,  2.95438196e-03,
        -2.06059571e-03, -1.52328563e-05],
       [ 3.30315669e-19, -1.47312288e-23, -3.24484487e-22,
        -1.07568783e-18,  2.76695240e-22, -6.66104018e-22,
         2.65557727e-22,  3.95980407e-24],
       [-6.94057995e-01, -6.82219787e-05, -3.85440714e-02,
        -3.00915084e-13,  3.11053091e-02, -6.34153829e-02,
         4.42303897e-02,  3.26971064e-04],
       [ 6.81991999e-01,  6.82228601e-05,  3.85445694e-02,
         3.00918972e-13, -3.11057110e-02,  6.34162022e-02,
        -4.42309612e-02, -3.26975288e-04],
       [-3.65281998e-01,  5.762334